user details - exploratory data analysis
===
# introduction
this notebook is solely for the purposes of getting to know the data and discovering its shapes and secrets. data cleaning and preprocessing takes place further downstream from here.

the data tables are:
1. profiles (housesitter profiles)
2. listings (homeowner listings)
3. pets (details of the pets belonging to a listing)
4. users (user contact details)
5. applications (requests by a sitter to a sit, and invites of homeowneres to sitters)
6. assignments (sitter + sits connections)
7. feedback (feedback left by housesitters on the sit)
8. reviews (feedback left by homeowners on the sitter)

In [6]:
from datetime import datetime
import duckdb
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import ydata_profiling as yp

DATAFOLDER='../data/raw/'        # path to raw data files
DATABASEFILE='../data/sits.ddb'  # path to persistent duckdb database file

In [9]:
np.iinfo(np.uint16).max

65535

# user contact details 
in this data set, each row represents user's details. from the data dictionary, we can learn a lot about the columns included. for instance that this table has no foerign keys, while . 

many of the columns are boolean, some are datetimes, others are strings, that can either be interpreted as categorical variables or harvested for one-hot encoded variables. 

## assumption
for each boolean variable, if an answer is missing, we will interpret that as FALSE, i.e. we take the question to mean whether _they are confirmed to be true_ or not. this assumption is not always going to be true, but it will be more often than not, and increases the expressiveness of the data set. 

In [49]:
bool_cols = [
    'can_contact_phone',
]

date_columns = [
    'expires_dt',
    'joined_dt',
    'registered_dt',
    'registered_ts',
]
today = pd.Timestamp.now().floor('D')  # floor to remove time component

category_cols=[
    'registration_method',
    'alerts',
    'billing_city',
    'billing_country',
    'reg_membership_type',
    'membership_status',
]

# numerical columns
data_types = {
    'id':np.uint32,
    'nb_raf_emails_sent':np.uint16,
    'nb_raf_referrals':np.uint16,
}

In [50]:
users_df =  (
    pd.read_csv(
        DATAFOLDER+'users.csv',
        low_memory=False,
        parse_dates=date_columns,
        date_format='%Y-%m-%d %H:%M:%S.%f',
        dtype=data_types,
    )
    .assign(
        registration_method  = lambda x: pd.Categorical(x['registration_method']),
        alerts               = lambda x: pd.Categorical(x['alerts']),
        billing_city         = lambda x: pd.Categorical(x['billing_city']),
        billing_state        = lambda x: pd.Categorical(x['billing_state']),
        billing_country      = lambda x: pd.Categorical(x['billing_country']),
        reg_membership_type  = lambda x: pd.Categorical(x['reg_membership_type']),
        membership_status    = lambda x: pd.Categorical(x['membership_status']),
        nb_raf_referrals     = lambda x: x['nb_raf_referrals'].astype('Int16'),
        
    )
    #     days_since_modified    = lambda x: (today - x['modified_dt']).dt.days.astype('Int16'),
    #     birth_decade           = lambda x: pd.Series(((x['birth_date'].dt.year//10)*10),dtype='Int16'),
    #     partner_birth_decade   = lambda x: pd.Series(((x['partner_birth_date'].dt.year//10)*10),dtype='Int16'),
    #     with_a_baby            = lambda x: x['child_ages'].str.contains('0-3').fillna(False),
    #     with_a_toddler         = lambda x: x['child_ages'].str.contains('4-7').fillna(False),
    #     with_a_child           = lambda x: x['child_ages'].str.contains('8-12').fillna(False),
    #     with_a_teen            = lambda x: x['child_ages'].str.contains('13-17|18+').fillna(False),
    #     dog_skills             = lambda x: x['skills'].str.contains('dog').fillna(False),
    #     cat_skills             = lambda x: x['skills'].str.contains('cat').fillna(False),
    #     five_star_ratio        = lambda x: (x['nb_5s_reviews']/x['nb_reviews']).fillna(0).astype(np.float32),
    #     can_give_medicine      = lambda x: x['skills'].str.contains('medication').fillna(False),
    #     big_dog_preferences    = lambda x: x['dog_size_preferences'].str.contains('^L|XL|,L').fillna(False),
    #     small_dog_preferences  = lambda x: x['dog_size_preferences'].str.contains('^S|XS|,S').fillna(False),
    #     years_of_experience    = lambda x: pd.to_numeric(x['years_of_experience'].fillna(0), errors='coerce', downcast='integer'),
    #     wish_list_city         = lambda x: x['location_wish_list'].str.contains('city').fillna(False),
    #     wish_list_beach        = lambda x: x['location_wish_list'].str.contains('beach').fillna(False),
    #     wish_list_mountain     = lambda x: x['location_wish_list'].str.contains('mountain').fillna(False),
    #     wish_list_countryside  = lambda x: x['location_wish_list'].str.contains('countryside').fillna(False),
    #     daily_minutes_willing_to_walk_dogs = lambda x: pd.to_numeric(x['daily_minutes_willing_to_walk_dogs'].fillna(0), errors='coerce', downcast='integer'),
)

# for booleans, if answer missing, assume it is 'no':
for col in bool_cols:
    users_df[col] = users_df[col].astype('bool').fillna(False)

display(users_df.sample(5))
display(users_df.info())

,alerts,billing_city,billing_country,billing_state,can_contact_phone,expires_dt,id,joined_dt,marketing_comms_accepted,membership_status,nb_raf_emails_sent,nb_raf_referrals,reg_membership_type,registered_dt,registered_ts,registration_method
17377,none,Warmington,united-kingdom,Oxfordshire,True,2025-09-06 00:00:00.000,2713207,2021-07-05,False,current member,0,0,homeowner,2021-06-22,2021-06-22 20:56:36,email
78209,daily,NaN,united-kingdom,NaN,True,2025-06-01 00:00:00.000,3293711,2022-02-10,True,current member,0,0,homeowner,2022-02-10,2022-02-10 09:41:40,google
38815,none,NaN,portugal,NaN,True,2023-10-29 00:00:00.000,4226660,2022-10-29,True,expired member,0,0,housesitter,2022-07-24,2022-07-24 13:22:27,google
16480,none,Colorado Springs,united-states,Colorado,True,2023-04-20 00:00:00.000,2610503,2021-04-20,True,expired member,0,0,housesitter,2021-04-20,2021-04-20 05:46:28,google
56977,none,NaN,united-kingdom,NaN,False,2024-12-23 00:00:00.000,402391,2022-07-04,False,expired member,0,0,housesitter,2016-12-24,2016-12-24 14:10:11,google


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104414 entries, 0 to 104413
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   alerts                    104414 non-null  category      
 1   billing_city              37152 non-null   category      
 2   billing_country           104411 non-null  category      
 3   billing_state             37120 non-null   category      
 4   can_contact_phone         104414 non-null  bool          
 5   expires_dt                104414 non-null  object        
 6   id                        104414 non-null  uint32        
 7   joined_dt                 104414 non-null  datetime64[ns]
 8   marketing_comms_accepted  104414 non-null  bool          
 9   membership_status         104414 non-null  category      
 10  nb_raf_emails_sent        104414 non-null  uint16        
 11  nb_raf_referrals          104414 non-null  Int16         
 12  re

None

In [64]:
# users_df['reg_membership_type'].unique()
display(users_df[users_df['expires_dt']>'2030-01-01'])
print(pd.Timestamp.min, '---', pd.Timestamp.max)

,alerts,billing_city,billing_country,billing_state,can_contact_phone,expires_dt,id,joined_dt,marketing_comms_accepted,membership_status,nb_raf_emails_sent,nb_raf_referrals,reg_membership_type,registered_dt,registered_ts,registration_method
1,real,London,united-kingdom,London,False,2121-03-01,148,2010-11-04,True,current member,0,0,combined,2012-06-22,2012-06-22 05:47:50,email
83,daily,Fjaltring,denmark,Lemvig,False,2117-12-29,12133,2013-01-03,True,current member,0,0,homeowner,2013-01-03,2013-01-03 18:43:13,facebook
175,daily,Hillarys,australia,Western Australia,True,2050-04-15,4603,2012-02-07,True,current member,0,0,combined,2012-02-07,2012-02-07 11:18:58,email
458,none,Crestview,united-states,FL,True,2075-06-22,21203,2013-09-23,False,current member,4,0,housesitter,2013-09-23,2013-09-23 05:04:15,google
629,daily,Elizabeth Bay,australia,New South Wales,False,2075-07-29,19108,2013-07-29,True,current member,0,0,housesitter,2013-07-29,2013-07-29 09:23:42,email
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71324,none,Myrtle Beach,united-states,SC,True,2031-02-04,2091070,2020-03-04,True,current member,0,0,housesitter,2020-02-16,2020-02-16 21:32:33,facebook
75475,daily,NaN,united-states,NaN,True,2030-11-27,3449384,2022-03-27,True,current member,0,0,homeowner,2022-03-20,2022-03-20 16:13:16,facebook
82246,none,NaN,united-kingdom,NaN,True,2126-12-15,3580413,2022-04-15,True,current member,0,0,housesitter,2022-04-15,2022-04-15 17:04:08,email
86179,none,NaN,united-states,NaN,True,2074-07-26,3809139,2022-05-26,True,current member,0,0,combined,2022-05-26,2022-05-26 18:47:10,email


1677-09-21 00:12:43.145224193 --- 2262-04-11 23:47:16.854775807


In [59]:
%%time
# convert timestamp strings to datetime, and bring extremes within bounds
def safe_date_parse(date_str):
    try:
        return pd.to_datetime(date_str)
    except:
        # extract date parts for out-of-bounds dates
        if isinstance(date_str, str):
            try:
                # year, month, day
                date_parts = date_str.split(' ')[0].split('-')
                if len(date_parts) == 3:
                    year, month, day = map(int, date_parts)
                    # when year is out of bounds, set it to maximum/minimum allowed
                    if year > 2262:
                        return pd.Timestamp.max
                    elif year < 1677:
                        return pd.Timestamp.min
            except:
                pass
        return pd.NaT
# slow, but works
users_df['expires_dt'] = users_df['expires_dt'].apply(safe_date_parse).dt.date


# exploratory data analysis reporting

In [65]:
profile = yp.ProfileReport(users_df)
profile.to_file('../eda_report_users.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/oh/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/ydata_profiling/model/pandas/describe_date_pandas.py:58: UserWarning: Discarding nonzero nanoseconds in conversion.
  "max": pd.Timestamp.to_pydatetime(series.max()),


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# lessons from exploratory analysis report
1. 
2. 
3. 

In [ ]:
con = duckdb.connect(DATABASEFILE)
con.sql("CREATE TABLE users_prep AS SELECT * FROM users_df")